In [23]:
from pathlib import Path
import numpy as np
import pandas as pd
import os 
import geemap
import ee
from typing import Dict, List, Tuple
from tqdm import tqdm
from utils import extractROI


ee.Authenticate()
ee.Initialize(project="virtual-rarity-426212-p6")
Map = geemap.Map()


LAI_BAND = 'Lai_500m'
FOREST_BAND = 'Percent_Tree_Cover'
GREEN_BAND = 'NDVI'
ZIPPED_FHAPEFILES_DIR = "../zipped_files"
START_DATE = ee.Date('2000-02-18')
END_DATE = ee.Date('2023-01-01')
ASSET_FODLER = "projects/virtual-rarity-426212-p6/assets/shapefiles"

# Each zipfile is FeatureCollection type asset



class BasinMetrics():
    
    def __init__(self, asset_fldr: str):
        self.roi_lst, self.basin_names = extractROI(asset_fldr)
        self.terrain = ee.Image("USGS/SRTMGL1_003") # contains one image
        
    def computeBasinArea(self, roi: ee.Geometry)->float:
        area_km2 = round(roi.area().getInfo()/1e6, 7)
        return area_km2
    
        
    def _computeOneElevationSlope(self, roi: ee.Geometry)->Tuple[Dict, Dict]:
        
        elev_stats = self.terrain.reduceRegion(
            reducer = ee.Reducer.mean().combine(
                reducer2 = ee.Reducer.minMax(),
                sharedInputs = True
            ), 
            geometry = roi, 
            scale = 30,
            maxPixels = 1e9
        ).getInfo()
        
        slope = ee.Terrain.slope(self.terrain)
        
        slope_stats = slope.reduceRegion(
            reducer = ee.Reducer.mean().combine(
                reducer2 = ee.Reducer.minMax(),
                sharedInputs = True
            ),
            geometry = roi,
            scale = 30,
            maxPixels = 1e9
        ).getInfo()
        
       
        return slope_stats, elev_stats
        
    def computeAllElevationSlopeStats(self):
        
        
        table = {"area_gages2": [], "elev_mean": [], "slope_mean": []}
        bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        
        for idx, roi in enumerate(tqdm(self.roi_lst, bar_format=bar_format)):
            
            if idx==5:
                break
            
            area_gages2 = self.computeBasinArea(roi)
            slope_stats, elev_stats = self._computeOneElevationSlope(roi)
            
            table['area_gages2'].append(area_gages2)
            table['slope_mean'].append(slope_stats['slope_mean'])
            table['elev_mean'].append(elev_stats['elevation_mean'])
            
        dataframe = pd.DataFrame(table, index=self.basin_names[:5])
        return dataframe
        




class Vegetation():
    
    def __init__(self, start_date: str, end_date: str, land_cover: str = 'MODIS/061/MOD15A2H'):
        
        self.start_date = start_date
        self.end_date = end_date 
        self.landcover_dataset = ee.ImageCollection(land_cover).\
                    filterDate(start_date, end_date).select(LAI_BAND)
                    
        self.forest_fraction = ee.ImageCollection("MODIS/006/MOD44B").\
                    filterDate('2000-03-05', '2020-03-05').select(FOREST_BAND)
                    
        self.green_fraction = ee.ImageCollection("MODIS/061/MOD13Q1").\
                    filterDate(start_date, end_date).select(GREEN_BAND)
        
        self.roi_lst, self.basin_names = extractROI(ASSET_FODLER)
        self.months  = ee.List.sequence(1, 12)
     
        
    
    def _compute_monthly_lai_stats(self, month: str)->ee.ImageCollection:
        
        monthlyLai = self.landcover_dataset.filter(ee.Filter.calendarRange(month, month, 'month'))
        
        monthly_mean_lai = monthlyLai.mean().rename('lai_mean')
        monthly_max_lai = monthlyLai.max().rename('lai_max') # max of a month across the years 
        monthly_min_lai = monthlyLai.min().rename('lai_min') # min of a month across the years 
        
        monthly_stats_lai = monthly_mean_lai.addBands([monthly_max_lai, monthly_min_lai])
        
        return monthly_stats_lai.set('month', month)
        
        
    def _computeOneLaiStats(self, roi: ee.Geometry)->Tuple[float, float]:
        
        monthly_lai_stats = self.landcover_dataset.\
                            fromImages(self.months.map(lambda month: self._compute_monthly_lai_stats(month)))
        
        
        clipped_lai_images =  monthly_lai_stats.map(lambda img: img.clip(roi))
        lai_mean_images = clipped_lai_images.select('lai_mean')
        
    
        max_lai_mean = lai_mean_images.reduce(ee.Reducer.max()).reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=roi,
                            scale=500,  
                            maxPixels=1e9
                            )
        
        min_lai_mean = lai_mean_images.reduce(ee.Reducer.min()).reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=roi,
                            scale=500,  
                            maxPixels=1e9
                            )
        
        num_max_lai_mean = max_lai_mean.getInfo()['lai_mean_max']
        num_min_lai_mean = min_lai_mean.getInfo()['lai_mean_min']
        mean_diff = num_max_lai_mean - num_min_lai_mean
        
        return round(num_max_lai_mean/4, 7), round(mean_diff/4,7)  # 31/8-day composite
    
    
    
    def _computeForestFractionStats(self, roi: ee.Geometry)->float:
        
        clipped_tree_cover = self.forest_fraction.map(lambda img: img.clip(roi))
        
        mean_tree_cover = clipped_tree_cover.reduce(ee.Reducer.mean()).reduceRegion(
                            reducer = ee.Reducer.mean(),
                            geometry=roi,
                            scale=250,
                            maxPixels=1e9
                          )
        
        return round(mean_tree_cover.getInfo()['Percent_Tree_Cover_mean']/100, 7)
    
    # aggregate green fractions for a month across the years 
    def _computeOneGreenFraction(self, month: ee.Number)->ee.Image:
        
        monthly_ndvi = self.green_fraction.filter(ee.Filter.calendarRange(month, month, 'month'))
        mean_monthly_ndvi = monthly_ndvi.mean().rename('mean_ndvi')
        return mean_monthly_ndvi.set('month', month)
    
    
    # TODO: Check everything!
    def _computeGreenFractionStats(self, roi: ee.Geometry) -> Tuple[float, float, float]:
        mean_monthly_ndvi = self.green_fraction.\
                            fromImages(self.months.map(lambda month: self._computeOneGreenFraction(month)))
        
        clipped_imgs = mean_monthly_ndvi.map(lambda img: img.clip(roi)).select('mean_ndvi') 
        
        # Reduce the ImageCollection to a single Image by calculating the mean
        mean_image = clipped_imgs.mean()
        
        # Combine reducers to calculate mean, min, and max
        combined_reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        )
        
        # Apply the combined reducer
        stats = mean_image.reduceRegion(
            reducer=combined_reducer,
            geometry=roi,
            scale=250,
            maxPixels=1e9
        ).getInfo()
        
        # Extract values and apply scaling factor
        scaling_factor = 0.0001
        mean_ndvi = stats['mean_ndvi_mean'] * scaling_factor
        min_ndvi = stats['mean_ndvi_min'] * scaling_factor
        max_ndvi = stats['mean_ndvi_max'] * scaling_factor
        
        gvf_max = (mean_ndvi - min_ndvi) / (max_ndvi - min_ndvi)
        gvf_diff = max_ndvi - min_ndvi
        
        return round(gvf_max, 7), round(gvf_diff, 7), round(mean_ndvi, 7)


        
    def computeAllStats(self):
        
        table = {"lai_max": [], "lai_diff": [], "forest_frac": [],
                 "gvf_max": [], "gvf_diff": []}
        
        bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        
        for idx, roi in enumerate(tqdm(self.roi_lst, bar_format=bar_format)):
            
            if idx==5:
                break
            lai_max, lai_diff = self._computeOneLaiStats(roi)
            forest_frac = self._computeForestFractionStats(roi)
            gvf_max, gvf_diff, _ = self._computeGreenFractionStats(roi)
            
            table['lai_max'].append(lai_max)
            table['lai_diff'].append(lai_diff)
            table['forest_frac'].append(forest_frac)
            table['gvf_max'].append(gvf_max)
            table['gvf_diff'].append(gvf_diff)
            
            
        dataframe = pd.DataFrame(table, index=self.basin_names[:5])
        return dataframe
    
   

    
# # vegi = Vegetation(START_DATE, END_DATE)
# basinMetric = BasinMetrics(ASSET_FODLER)
# result1 = basinMetric.computeAllElevationSlopeStats()
# result1.head()



  5%|▍         | 5/106 [00:08<02:45,  1.64s/it]


,area_gages2,elev_mean,slope_mean
11001,58489.499676,1473.256087,12.473454
11063,97.984369,1176.012101,11.648020
11068,2953.293899,1429.457014,10.788666
11077,2521.671927,1449.100238,7.657305
11090,222.672868,593.480450,2.310260
